ref: https://qiita.com/XBRLJapan/items/d23bc251c53d81d49852#1-%E3%81%AF%E3%81%98%E3%82%81%E3%81%AB
        

In [89]:
import glob
from pathlib import Path
import re

In [95]:
# XBRLのパスから、有価証券報告書内のテキスト情報をデータフレームで返す関数本体。
def get_nonnumeric(arg_path):
    # fsファイルの読み込み。bs4でパース
    with open(arg_path, encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'lxml')

    # nonNumericタグのみ抽出
    tags_nonnumeric = soup.find_all('ix:nonnumeric')

    # nonnumericの各要素を格納するカラのリストを作成
    list_nonnumeric = []
    # nonnumericの内容を辞書型に
    for tag in tags_nonnumeric:
        dict_tag = {}
        dict_tag['nonnumeric_tag'] = tag.get('name')
        dict_tag['nonnumeric_text'] = tag.text
        list_nonnumeric.append(dict_tag)

    # 辞書を格納したリストをDFに
    df_nonnumeric = pd.DataFrame(list_nonnumeric)

    return df_nonnumeric

# XBRLのパスから、有価証券報告書内の数値情報をデータフレームで返す関数本体。
def get_nonfraction(arg_path):
    # fsファイルの読み込み。bs4でパース
    with open(arg_path, encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'lxml')

    # nonFractionタグのみ抽出
    tags_nonfraction = soup.find_all('ix:nonfraction')

    # nonfractionの各要素を格納するカラの辞書を作成
    list_nonfraction = []
    # nonfractionの内容を辞書型に
    # sign : 符号
    # scale: 10の冪乗の数
    for tag in tags_nonfraction:
        dict_fs = {}
        dict_fs['name'] = tag.get('name')
        dict_fs['contextRef'] = tag.get('contextref')
#         dict_fs['format'] = tag.get('format')
#         dict_fs['decimals'] = tag.get('decimals')
        dict_fs['scale'] = tag.get('scale')
        
#         dict_fs['unitRef'] = tag.get('unitRef')
        # マイナス表記の場合の処理＋円単位への変更
#         if tag.get('sign') == '-' and tag.get('xsi:nil') != 'true':
#             amount = int(float(tag.text.replace(',', ''))) * -1 * 10 ** int(tag.get('scale'))
#         elif tag.get('xsi:nil') != 'true':
#             amount = int(float(tag.text.replace(',', ''))) * 10 ** int(tag.get('scale'))
#         else:
#             amount = ''
        amount = tag.text
        dict_fs['amount'] = amount
        dict_fs["sign"] = tag.get("sign")
        # 辞書をリストへ格納
        list_nonfraction.append(dict_fs)

    # 辞書を格納したリストをDFに
    df_nonfraction = pd.DataFrame(list_nonfraction)

    return df_nonfraction

### タグについて
企業によって、営業収益　や　売上高　など語彙が違うことがある。（企業側でタグをカスタムできるらしいので、nameを変えられていた場合は取れない可能性がある）

#### 語彙と対応するname
売上高：NetSales<br>
営業収益：OperatingRevenuesREIT<br>
営業収益：OperatingRevenuesSE<br>
純営業収益：NetOperatingRevenuesSE<br>
営業利益：OperatingIncome<br>
経常利益：OrdinaryIncome<br>
親会社株主に帰属する四半期純利益: ProfitAttributableToOwnersOfParent<br>
純利益：NetIncome<br>
四半期純利益：NetIncome　かつ、　contextRefがCurrentAccumulatedQ2Duration・・・・<br>
当期純利益：NetIncome　かつ、　CurrentYearDuration_NonConsolid・・・・・<br>
2022年２月期の予想：contextRefがNextYearDuration_NonConsolidatedMember_Foreca<br>
2022年8月期の予想：contextRefがNext2YearDuration_NonConsolidatedMembe...<br>
上二つのようなデータはfintosの対象になる？<br>
通期予想：contextRefがCurrentYearDuration_ConsolidatedMember_Forec<br>

増減は、ChangeIn〇〇<br>

In [96]:
# 拾うタグを正規表現で定義

# 営業利益
OI_pattern = re.compile("tse-[a-z]{2}-t:OperatingIncome")
# 営業利益の増減
Change_In_OI_pattern = re.compile("tse-[a-z]{2}-t:ChangeInOperatingIncome")

In [97]:
xbrl_path = "/Users/takahashimichika/Documents/KessanTanshin/sample_XBRL/*"

In [103]:
folders = glob.glob(xbrl_path)
for folder in folders:
    summary_folder = os.path.join(folder + "/Summary")
    target= glob.glob(os.path.join(summary_folder + "/*.htm"))[0]

    ff = open(target , "r" ,encoding="utf-8" ).read() 
    soup = BeautifulSoup( ff ,"html.parser")
    
    nms = soup.find_all("ix:nonnumeric")
    for nm in nms[0:5]:
        print(nm.text)
        
    df = get_nonfraction(target)
    
    eigyorieki = df["amount"][(df["name"].str.match(OI_pattern)) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ResultMember"))]
    eigyorieki_diff = df["amount"][(df["name"].str.match(Change_In_OI_pattern)) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ResultMember"))]
    eigyorieki_diff_pn = df["sign"][(df["name"].str.match(Change_In_OI_pattern)) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ResultMember"))]
    if len(eigyorieki) != 0:
        print("営業利益：　",eigyorieki.iloc[-1],"百万円")
        if eigyorieki_diff_pn.iloc[-1] != None:
            print("増減：　-",eigyorieki_diff.iloc[-1],"%")    
        else:
            print("増減：　",eigyorieki_diff.iloc[-1],"%")        
    tsuki_eigyorieki = df["amount"][(df["name"].str.match(OI_pattern )) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ForecastMember"))]
    tsuki_eigyorieki_diff = df["amount"][(df["name"].str.match(Change_In_OI_pattern)) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ForecastMember"))]
    tsuki_eigyorieki_diff_pn = df["sign"][(df["name"].str.match(Change_In_OI_pattern)) & (df["contextRef"].str.contains("Current")) & (df["contextRef"].str.contains("ForecastMember"))]
    if len(tsuki_eigyorieki) != 0:
        print("通期営業利益：　",tsuki_eigyorieki.iloc[-1],"百万円")
        if tsuki_eigyorieki_diff_pn.iloc[-1] != None:
            print("増減：　-",tsuki_eigyorieki_diff.iloc[-1],"%")    
        else:
            print("増減：　",tsuki_eigyorieki_diff.iloc[-1],"%")    
    else:
        print("通期見通し開示なし")

    print("\n")


第3四半期決算短信〔日本基準〕（連結）
2021年10月25日
キヤノン電子株式会社
東
営業利益：　 3,745 百万円
増減：　- 0.3 %
通期営業利益：　 7,975 百万円
増減：　 42.3 %


決算短信（ＲＥＩＴ）
2021年10月19日
日本アコモデーションファンド投資法人
東

営業利益：　 5,428 百万円
増減：　 3.1 %
通期見通し開示なし



第２四半期決算短信〔日本基準〕（連結）
2021年10月20日
アルインコ株式会社
東
営業利益：　 922 百万円
増減：　- 5.5 %
通期営業利益：　 2,990 百万円
増減：　 17.0 %



第2四半期決算短信〔日本基準〕（連結）
2021年10月21日
株式会社 ナガセ
東
営業利益：　 1,653 百万円
増減：　  %
通期営業利益：　 5,962 百万円
増減：　 29.8 %



第2四半期決算短信〔日本基準〕（非連結）
2021年10月21日
光世証券株式会社
東
営業利益：　 159 百万円
増減：　  %
通期見通し開示なし




In [84]:
df = get_nonfraction("/Users/takahashimichika/Documents/KessanTanshin/sample_XBRL/XBRLData 3/Summary/tse-qcedjpsm-97330-20211009408732-ixbrl.htm")
display(df[:65])
print(df.loc[0]["contextRef"])

,name,contextRef,scale,amount,sign
0,tse-ed-t:NetSales,CurrentAccumulatedQ2Duration_ConsolidatedMembe...,6,"22,128",None
1,tse-ed-t:ChangeInNetSales,CurrentAccumulatedQ2Duration_ConsolidatedMembe...,-2,19.6,None
2,tse-ed-t:OperatingIncome,CurrentAccumulatedQ2Duration_ConsolidatedMembe...,6,"1,653",None
3,tse-ed-t:ChangeInOperatingIncome,CurrentAccumulatedQ2Duration_ConsolidatedMembe...,None,,None
4,tse-ed-t:OrdinaryIncome,CurrentAccumulatedQ2Duration_ConsolidatedMembe...,6,"1,477",None
...,...,...,...,...,...
60,tse-ed-t:OrdinaryIncome,CurrentYearDuration_ConsolidatedMember_Forecas...,6,"5,602",None
61,tse-ed-t:OrdinaryIncome,CurrentYearDuration_ConsolidatedMember_LowerMe...,None,,None
62,tse-ed-t:OrdinaryIncome,CurrentYearDuration_ConsolidatedMember_UpperMe...,None,,None
63,tse-ed-t:ChangeInOrdinaryIncome,CurrentYearDuration_ConsolidatedMember_Forecas...,-2,24.2,None


CurrentAccumulatedQ2Duration_ConsolidatedMember_ResultMember
